In [ ]:
 url = "https://www.airnowapi.org/aq/data/?startDate=2023-10-08T13&endDate=2023-12-09T14&parameters=CO&BBOX=-118.951721,32.75004,-117.646374,34.823302&dataType=C&format=text/csv&verbose=0&monitorType=0&includerawconcentrations=0&API_KEY=8F7089D2-5912-49EE-8EEA-918600DADD1C"

In [ ]:
import requests
s = requests.get(url).content

In [ ]:
import pandas as pd
import io
column_name = ['latitude', 'longitude', 'utc', 'parameter', 'value', 'unit']
series = pd.read_csv(io.StringIO(s.decode('utf-8')), names=column_name, usecols=column_name)

In [ ]:
series.head(-5)

,latitude,longitude,utc,parameter,value,unit
0,34.199200,-118.533100,2023-10-08 13:00:00,CO,0.6,PPM
1,34.383300,-118.528300,2023-10-08 13:00:00,CO,0.3,PPM
2,34.066429,-118.226755,2023-10-08 13:00:00,CO,0.7,PPM
3,33.901400,-118.205000,2023-10-08 13:00:00,CO,0.5,PPM
4,34.132630,-118.127142,2023-10-08 13:00:00,CO,0.3,PPM
...,...,...,...,...,...,...
16171,34.383300,-118.528300,2023-12-09 14:00:00,CO,0.1,PPM
16172,34.066429,-118.226755,2023-12-09 14:00:00,CO,0.7,PPM
16173,33.901400,-118.205000,2023-12-09 14:00:00,CO,1.2,PPM
16174,34.132630,-118.127142,2023-12-09 14:00:00,CO,0.4,PPM


In [ ]:
series['utc'] = pd.to_datetime(series['utc'])

In [1]:
df_daily = series.groupby(series['utc'].dt.date).mean()

NameError: ignored

In [ ]:
df_daily.reset_index(inplace=True)

In [ ]:


df_daily = df_daily.dropna()

In [ ]:
 df_daily['value'] = df_daily['value'].astype(float)

In [ ]:
data = df_daily['value']

In [ ]:
data = data.values

In [ ]:
history_original = [data[-60:]]

In [ ]:
history_original

[array([0.23058824, 0.33266932, 0.39224806, 0.45189394, 0.49659091,
        0.55378788, 0.48710938, 0.50790514, 0.5390625 , 0.54734848,
        0.48106061, 0.31931818, 0.20381679, 0.32348485, 0.23217054,
        0.26136364, 0.3878327 , 0.41477273, 0.3594697 , 0.38219697,
        0.42895753, 0.51633466, 0.5519084 , 0.67607843, 0.69962121,
        0.65378788, 0.65151515, 0.26780303, 0.36769231, 0.4859375 ,
        0.51856061, 0.6469697 , 0.525     , 0.58409091, 0.60703125,
        0.66870229, 0.28242187, 0.41589147, 0.24962121, 0.24166667,
        0.32689394, 0.37382813, 0.36564885, 0.50984848, 0.24469697,
        0.32878788, 0.28674242, 0.31666667, 0.4835249 , 0.5003876 ,
        0.38455598, 0.33977273, 0.42765152, 0.55265152, 0.7       ,
        0.70465116, 0.73574297, 0.3751938 , 0.39527559, 0.54      ])]

In [ ]:
def normalize_windows(window_data):

    """Normalize data"""
    normalized_data = []
    for window in window_data:
        normalized_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalized_data.append(normalized_window)
    return normalized_data

In [ ]:
 history = normalize_windows(history_original)

In [ ]:
import numpy as np
history = np.array(history)

In [ ]:
 history = np.reshape(history, (history.shape[0], history.shape[1],1))

In [ ]:
history

array([[[ 0.        ],
        [ 0.44269859],
        [ 0.70107578],
        [ 0.95974412],
        [ 1.15358302],
        [ 1.40163111],
        [ 1.11246413],
        [ 1.20264983],
        [ 1.33777105],
        [ 1.37370516],
        [ 1.08623222],
        [ 0.38479824],
        [-0.11610064],
        [ 0.40286797],
        [ 0.00686205],
        [ 0.13346475],
        [ 0.68192752],
        [ 0.79875928],
        [ 0.55892471],
        [ 0.65748686],
        [ 0.860275  ],
        [ 1.23920644],
        [ 1.39348029],
        [ 1.93197279],
        [ 2.03407158],
        [ 1.83530458],
        [ 1.82544836],
        [ 0.16139069],
        [ 0.59458399],
        [ 1.10738202],
        [ 1.24885977],
        [ 1.80573593],
        [ 1.27678571],
        [ 1.53304731],
        [ 1.63253348],
        [ 1.89998442],
        [ 0.22478874],
        [ 0.80361098],
        [ 0.08254097],
        [ 0.04804422],
        [ 0.41765229],
        [ 0.6211934 ],
        [ 0.58572208],
        [ 1

In [ ]:
from keras.models import load_model
loaded_model = load_model('Los-Angeles-CO.keras')

In [ ]:
prediction = loaded_model.predict(history)

1/1 [==============================] - 0s 17ms/step


In [ ]:
prediction = np.reshape(prediction,(prediction.size,))

In [ ]:
prediction

array([1.2493392], dtype=float32)

In [ ]:
pollution = (prediction[0] + 1) * history_original[0][0]

In [ ]:
pollution

0.5186711619882023

In [ ]:
history_original[0]

array([0.23058824, 0.33266932, 0.39224806, 0.45189394, 0.49659091,
       0.55378788, 0.48710938, 0.50790514, 0.5390625 , 0.54734848,
       0.48106061, 0.31931818, 0.20381679, 0.32348485, 0.23217054,
       0.26136364, 0.3878327 , 0.41477273, 0.3594697 , 0.38219697,
       0.42895753, 0.51633466, 0.5519084 , 0.67607843, 0.69962121,
       0.65378788, 0.65151515, 0.26780303, 0.36769231, 0.4859375 ,
       0.51856061, 0.6469697 , 0.525     , 0.58409091, 0.60703125,
       0.66870229, 0.28242187, 0.41589147, 0.24962121, 0.24166667,
       0.32689394, 0.37382813, 0.36564885, 0.50984848, 0.24469697,
       0.32878788, 0.28674242, 0.31666667, 0.4835249 , 0.5003876 ,
       0.38455598, 0.33977273, 0.42765152, 0.55265152, 0.7       ,
       0.70465116, 0.73574297, 0.3751938 , 0.39527559, 0.54      ])